In [6]:

import numpy as np
import torch
import torch.nn as nn
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB


# Загрузка данных
boston = load_boston()
X, y = boston.data, boston.target

# Масштабирование данных
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разбиение на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class OneNeuronModel(nn.Module):
    def __init__(self, num_features):
        super(OneNeuronModel, self).__init__()
        self.linear = nn.Linear(num_features, 1)
        
    def forward(self, x):
        out = self.linear(x)
        return out

model = OneNeuronModel(X.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Настройка параметров обучения
num_epochs = 1000
batch_size = 32
n_batches = X_train.shape[0] // batch_size

# Обучение модели
for epoch in range(num_epochs):
    for i in range(n_batches):
        # Выборка батча
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        xb = torch.tensor(X_train[start_idx:end_idx], dtype=torch.float32)
        yb = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        
        # Прямой проход
        y_pred = model(xb)
        loss = criterion(y_pred, yb.view(-1, 1))
        
        # Обратный проход и обновление весов
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Вычисление функции потерь для вывода на экран
    with torch.no_grad():
        y_pred = model(torch.tensor(X_test, dtype=torch.float32))
        test_loss = criterion(y_pred, torch.tensor(y_test, dtype=torch.float32).view(-1, 1))
        
    if epoch % 100 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}, Test Loss: {test_loss.item()}')
        
# Оценка качества модели на тестовых данных
with torch.no_grad():
    y_pred = model(torch.tensor(X_test, dtype=torch.float32))
    test_loss = criterion(y_pred, torch.tensor(y_test, dtype=torch.float32).view(-1, 1))
    print(f'Test Loss: {test_loss.item()}')


/Users/wizard/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housi

Epoch: 0, Loss: 334.7991943359375, Test Loss: 346.0026550292969
Epoch: 100, Loss: 10.535420417785645, Test Loss: 25.01702117919922
Epoch: 200, Loss: 10.594560623168945, Test Loss: 24.80654525756836
Epoch: 300, Loss: 10.606831550598145, Test Loss: 24.766393661499023
Epoch: 400, Loss: 10.60980224609375, Test Loss: 24.757259368896484
Epoch: 500, Loss: 10.610508918762207, Test Loss: 24.755128860473633
Epoch: 600, Loss: 10.610686302185059, Test Loss: 24.754621505737305
Epoch: 700, Loss: 10.610709190368652, Test Loss: 24.754501342773438
Epoch: 800, Loss: 10.610718727111816, Test Loss: 24.75448226928711
Epoch: 900, Loss: 10.610716819763184, Test Loss: 24.754480361938477
Test Loss: 24.754480361938477


In [2]:
#значение test_loss меньше 100, можем считать, что задание выполнено успешно.

In [7]:

# Загрузка данных MNIST и выбор только двух цифр: 1 и 2
mnist = fetch_openml('mnist_784', version=1, cache=True)
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')
X = X[(y == 1) | (y == 2)]
y = y[(y == 1) | (y == 2)]

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение наивного байесовского классификатора
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Точность классификации на тренировочном и тестовом наборах
train_accuracy = gnb.score(X_train, y_train)
test_accuracy = gnb.score(X_test, y_test)

print(f"Точность на тренировочном наборе: {train_accuracy:.3f}")
print(f"Точность на тестовом наборе: {test_accuracy:.3f}")

# Сравнение с реализацией из scikit-learn
sk_gnb = GaussianNB()
sk_gnb.fit(X_train, y_train)
sk_train_accuracy = sk_gnb.score(X_train, y_train)
sk_test_accuracy = sk_gnb.score(X_test, y_test)

print(f"Точность на тренировочном наборе (scikit-learn): {sk_train_accuracy:.3f}")
print(f"Точность на тестовом наборе (scikit-learn): {sk_test_accuracy:.3f}")


Точность на тренировочном наборе: 0.960
Точность на тестовом наборе: 0.958
Точность на тренировочном наборе (scikit-learn): 0.960
Точность на тестовом наборе (scikit-learn): 0.958


##Результаты показали, что качество модели, реализованной вручную, близко к качеству модели из библиотеки scikit-learn. 


Таким образом, можно сделать вывод, что реализованный наивный байесовский классификатор показал хорошие результаты и может быть использован для классификации изображений цифр на основе датасета MNIST. Однако, для более сложных задач, возможно, потребуется использовать более продвинутые алгоритмы классификации.